In [1]:
from PIL import Image,ImageDraw
import os,sys
def readfile(filename):
    with open(filename, "r") as ins:
        lines=[line for line in ins]
    # First line is the column titles
    #print(lines)
    colnames=lines[0].strip( ).split('\t')[1:]
    rownames=[]
    data=[]
    for line in lines[1:]:
        p=line.strip( ).split('\t')
        #print(p)
        #break
        # First column in each row is the rowname
        rownames.append(p[0])
        #print(rownames)
        # The data for this row is the remainder of the row
        try:
            data.append([float(x) for x in p[1:]])
        except ValueError :
            print("error")
       
    return rownames,colnames,data

In [2]:
from math import sqrt
def pearson(v1,v2):
    # Simple sums
    sum1=sum(v1)
    sum2=sum(v2)
    # Sums of the squares
    sum1Sq=sum([pow(v,2) for v in v1])
    sum2Sq=sum([pow(v,2) for v in v2])
    # Sum of the products
    pSum=sum([v1[i]*v2[i] for i in range(len(v1))])
    # Calculate r (Pearson score)
    num=pSum-(sum1*sum2/len(v1))
    den=sqrt((sum1Sq-pow(sum1,2)/len(v1))*(sum2Sq-pow(sum2,2)/len(v1)))
    if den==0: return 0
    return 1.0-num/den

In [3]:
class bicluster:
    def __init__(self,vec,left=None,right=None,distance=0.0,id=None):
        self.left=left
        self.right=right
        self.vec=vec
        self.id=id
        self.distance=distance
        
def hcluster(rows, distance=pearson):
    distances = {}
    currentclustid = -1

  # Clusters are initially just the rows
    clust = [bicluster(rows[i], id = i) for i in range(len(rows))]

    while len(clust) > 1:
        lowestpair = (0, 1)
        closest = distance(clust[0].vec, clust[1].vec)

    # loop through every pair looking for the smallest distance
        for i in range(len(clust)):
            for j in range(i + 1, len(clust)):
        # distances is the cache of distance calculations
                if (clust[i].id, clust[j].id) not in distances:
                    distances[(clust[i].id, clust[j].id)] = \
                        distance(clust[i].vec, clust[j].vec)
                d = distances[(clust[i].id, clust[j].id)]

                if d < closest:
                    closest = d
                    lowestpair = (i, j)
    # calculate the average of the two clusters
        mergevec = [(clust[lowestpair[0]].vec[i] + clust[lowestpair[1]].vec[i])
                    / 2.0 for i in range(len(clust[0].vec))]

    # create the new cluster
        newcluster = bicluster(mergevec, left=clust[lowestpair[0]],
                               right=clust[lowestpair[1]], distance=closest,
                               id=currentclustid)

    # cluster ids that weren't in the original set are negative
        currentclustid -= 1
        del clust[lowestpair[1]]
        del clust[lowestpair[0]]
        clust.append(newcluster)

    return clust[0]

In [4]:
blognames,word,data=readfile('blogdata5.txt')

error


In [7]:
clust=hcluster(data)

In [8]:
def printclust(clust,labels=None,n=0):
    # indent to make a hierarchy layout
    s=" "
    for i in range(n):
        s=s+"  "
    if clust.id<0:
        # negative id means that this is branch
        print (s+'-')
    else:
        # positive id means that this is an endpoint
        if labels==None: 
            print(s+ clust.id)
        else:
            
            print(s+ labels[clust.id])
    
    # now print the right and left branches
    if clust.left!=None:
        printclust(clust.left,labels=labels,n=n+1)
    if clust.right!=None: 
        printclust(clust.right,labels=labels,n=n+1)

In [35]:
printclust(clust,labels=blognames)

 -
   The Full Feed from HuffingtonPost.com
   -
     Quick Online Tips
     -
       Celebslam
       -
         PaulStamatiou.com - Technology, Design and Photography
         -
           The Write News
           -
             -
               Autoblog
               Make: DIY Projects and Ideas for Makers
             -
               NewsBusters
               -
                 -
                   SpikedHumor - Today's Videos and Pictures
                   -
                     Gigaom
                     -
                       Joel on Software
                       -
                         -
                           Lifehack
                           ProBlogger
                         -
                           John Battelle's Search Blog
                           -
                             Gapingvoid
                             -
                               O'Reilly Radar
                               -
                                 Tech
           

In [5]:
def getheight(clust):
  # Is this an endpoint? Then the height is just 1
    if clust.left == None and clust.right == None:
        return 1
  # Otherwise the height is the same of the heights of
  # each branch
    return getheight(clust.left) + getheight(clust.right)

In [6]:
def getdepth(clust):
    # The distance of an endpoint is 0.0
    if clust.left==None and clust.right==None: return 0
    # The distance of a branch is the greater of its two sides
    # plus its own distance
    return max(getdepth(clust.left),getdepth(clust.right))+clust.distance

In [7]:
def drawdendrogram(clust,labels,jpeg='clusters.jpg'):
    # height and width
    h=getheight(clust)*20
    w=1200
    depth=getdepth(clust)
    
    # width is fixed, so scale distances accordingly
    scaling=float(w-150)/depth
    
    # Create a new image with a white background
    img=Image.new('RGB',(w,h),(255,255,255))
    draw=ImageDraw.Draw(img)
    draw.line((0,h/2,10,h/2),fill=(255,0,0))
    
    # Draw the first node
    drawnode(draw,clust,10,(h/2),scaling,labels)
    img.save(jpeg,'JPEG')

In [24]:
def drawnode(draw,clust,x,y,scaling,labels):
    if clust.id < 0:
        h1 = getheight(clust.left) * 20
        h2 = getheight(clust.right) * 20
        top = y - (h1 + h2) / 2
        bottom = y + (h1 + h2) / 2
    # Line length
        ll = clust.distance * scaling
    # Vertical line from this cluster to children
        draw.line((x, top + h1 / 2, x, bottom - h2 / 2), fill=(255, 0, 0))

    # Horizontal line to left item
        draw.line((x, top + h1 / 2, x + ll, top + h1 / 2), fill=(255, 0, 0))

    # Horizontal line to right item
        draw.line((x, bottom - h2 / 2, x + ll, bottom - h2 / 2), fill=(255, 0,
                  0))

    # Call the function to draw the left and right nodes
        drawnode(
            draw,
            clust.left,
            x + ll,
            top + h1 / 2,
            scaling,
            labels
            )
        drawnode(
            draw,
            clust.right,
            x + ll,
            bottom - h2 / 2,
            scaling,
            labels
            )
    else:
    # If this is an endpoint, draw the item label
        print("-> ",labels[clust.id])
        draw.text((x + 5, y - 7),labels[ clust.id], (0, 0, 0))

In [39]:
drawdendrogram(clust,blognames,jpeg='blogclust.jpg')

->  The Full Feed from HuffingtonPost.com
->  Quick Online Tips
->  Celebslam
->  PaulStamatiou.com - Technology, Design and Photography
->  The Write News
->  Autoblog
->  Make: DIY Projects and Ideas for Makers
->  NewsBusters
->  SpikedHumor - Today's Videos and Pictures
->  Gigaom
->  Joel on Software
->  Lifehack
->  ProBlogger
->  John Battelle's Search Blog
->  Gapingvoid
->  O'Reilly Radar
->  Tech
->  PerezHilton
->  Little Green Footballs
->  Power LinePower Line
->  MichelleMalkin.com
->  Schneier on Security
->  ReadWrite
->  Copyblogger
->  Seth Godin's Blog on marketing, tribes and respect
->  Google Operating System
->  Search Engine Roundtable
->  mezzoblue
->  The Superficial - Because You're Ugly
->  ongoing by Tim Bray
->  456 Berea Street
->  Creating Passionate Users
->  plasticbag.org
->  Steve Pavlina
->  Guy Kawasaki
->  BuzzMachine
->  ScienceBlogs
->  Blog News
->  Instapundit
->  ShoeMoney
->  Tech Blog
->  Cool Hunting
->  Deadspin
->  Captain's Quarters
-> 

In [8]:
def rotatematrix(data):
    newdata=[]
    for i in range(len(data[0])):
        newrow=[data[j][i] for j in range (len(data))]
        newdata.append(newrow)
    return newdata

In [9]:
rdata=rotatematrix(data)

In [10]:
rcluster=hcluster(rdata)

KeyboardInterrupt: 

In [18]:
import random
    
def kcluster(rows,distance=pearson,k=4):
    
    # Determine the minimum and maximum values for each point
    #minimum value of each word and maximum value of each word
    ranges=[(min([row[i] for row in rows]),max([row[i] for row in rows]))
    for i in range(len(rows[0]))]
    
    
    # Create k randomly placed centroids
    clusters=[[random.random( )*(ranges[i][1]-ranges[i][0])+ranges[i][0]
    for i in range(len(rows[0]))] for j in range(k)]
    
    lastmatches=None
    for t in range(100):
        print( 'Iteration %d' % t)
        bestmatches=[[] for i in range(k)]
        
        # Find which centroid is the closest for each row
        for j in range(len(rows)):  #for each row
            row=rows[j]
            bestmatch=0             #let bestmatch be 0
            for i in range(k):       #for each cluster
                d=distance(clusters[i],row)     #find distance between cluster and row
                if d<distance(clusters[bestmatch],row): bestmatch=i      #find cluster witth lowest distance
            bestmatches[bestmatch].append(j)      #append the row number with the best closest cluster
            
            
        # If the results are the same as last time, this is complete
        if bestmatches==lastmatches: break
        lastmatches=bestmatches
        
        
        
        # Move the centroids to the average of their members
        for i in range(k):      #for every centroid
            avgs=[0.0]*len(rows[0])     #creating array list for number of columns in a row
            if len(bestmatches[i])>0:    #check if any row is assigned to a cluster
                for rowid in bestmatches[i]:      #for every rowid in bestmatches
                    for m in range(len(rows[rowid])):    #for every column in that row
                        avgs[m]+=rows[rowid][m]           #add value in avg in that column id
                for j in range(len(avgs)):          #find avg of that by diving eacg value in cloumn by the total number of clusters
                    avgs[j]/=len(bestmatches[i])   
                clusters[i]=avgs    #reassign the cluster
    return bestmatches
    

In [20]:
kclust=kcluster(data,k=10)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5


In [21]:
kclust

[[1, 11, 13, 16, 26, 27, 28, 51, 52, 53, 55, 56, 65, 67, 69],
 [],
 [29],
 [0, 4, 9, 10, 22, 39, 48, 62, 64, 73],
 [3, 6, 7, 15, 19, 20, 23, 24, 31, 34, 37, 40, 46, 60, 66, 70, 71],
 [43],
 [],
 [2, 36, 47, 50, 57, 58],
 [41],
 [5,
  8,
  12,
  14,
  17,
  18,
  21,
  25,
  30,
  32,
  33,
  35,
  38,
  42,
  44,
  45,
  49,
  54,
  59,
  61,
  63,
  68,
  72]]